## 노트북 내용

이 노트북은 watsonx.ai를 사용한 RAG (Retrieval Augmented Generation)을 시연하기 위한 것입니다. 여기에는 data retrieval, 지식 기본 정보에서 유사도검색 및 모델이 생성한 결과를 확인하는 내용이 포함되어 있습니다.
이를 잘 이해하기 위해서는 Python에 대한 기본 지식이 필요하며 모든 코드는 Python 3.10 으로 작성되어 있습니다.

### RAG (Retrieval Augmented Generation) 
Retrieval Augmented Generation (RAG)는 자연어로 지식 기반 데이터베이스에 질문이나 사실적인 정보를 사용하기를 원하는 다양한 usecase에 활용가능한 패턴입니다.

이 예제는 RAG의 가장 단순한 형태로서 세 가지 단계로 구성되어 있습니다. 

- 텍스트 데이타를 passage로 나누고 embedding하여 지식 기반 데이터베이스 구축
- 지식 기반 데이터베이스로부터 사용자 질문과 가장 유사한 passage들 추출
- 추출된 passage들을 large language model에 입력하여 사용자 질의에 대한 최종 답변 생성.

## 내용

이 노트북은 다음과 같은 단계로 구성되어 있습니다ㅏ.

- [환경설정](#setup)
- [지식기반 데이터베이스 구축](#build_base)
- [watsonx의 foundation model 접근 설정](#models)
- [사용자 질문에 대한 답변 생성](#predict)

<a id="setup"></a>
## 환경 설정

이노트북에 있는 샘플 코드를 실행하기 전에 다음 작업을 완료해야 합니다.

- 필요한 python package는 conda environment 혹은 python virtual environment에 python 3.10.12 기반의 독립적인 환경을 만든 후 pip install -r requirements_cp4d.txt를 사용해서 설치.
- Cloud Pak for Data 관리자에게 이 시스템에 접근할 수 있는 권한 정보를 획득하세요.



In [ ]:
!pip install "langchain-community"
!pip install "langchain"
!pip install "sentence-transformers"
!pip install "chromadb"
!pip install "pydantic"
!pip install -U "langchain-huggingface"
!pip install "ibm-watson-machine-learning==1.0.353"
!pip install "ibm_watsonx_ai==0.2.3"

In [22]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import WatsonxLLM

### Watson Machine Learning에 연결 정보 확인

Cloud Pak for Data에서 제공하는 Watson Machine Learning 서비스에 접근하기 위한 인증 정보를 입력한다.
여기에는 `url`, `username` 그리고 `api_key`가 포함된다.

In [ ]:
load_dotenv()

In [23]:
wml_credentials = {
    "username": 'jihunkim',
    "apikey": os.getenv("API_KEY", None),
    "url": 'https://cpd-watsonx.apps.elskr.zanity.net',
    "instance_id": 'openshift',
    "version": '4.8'
}

### project id 확인
Foundation model에 접근하기 위한 Cloud Pak for Data의 project id를 확인 한다.</br>
이 노트북이 Foundation model을 제공하는 Cloud Pak for Data이 Project 내에서 실행되는 경우 자동으로 환경 변수에서 가져올 수 있으나 Cloud Pak for Data 외부 환경에서 실행되는 경우는 연관된 project id를 확인한 후에 입력해 줘야 한다.

In [24]:
try:
    project_id = os.environ["PROJECT_ID"]
    run_in_cp4d = True
except KeyError:
    run_in_cp4d = False
    project_id = "Your Project ID"

<a id="build_base"></a>
## 기본 지식 정보(knowledge base) 구축

The current state-of-the-art in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.
최신 기술인 RAG는 사용자 질의에 대한 sematic similarity를 계산하기 위해 기본 지식 정보에 대한 밀집 벡터 표현 (dense vector represenations)를 생성한다.
이 기본 예제에서는 "영화 분노의 도로-퓨리오사 사가"의 줄거리를 가져와서 chunk로 나누고 embedding 한 후에 Chroma db에 저장하는 것으로 기본 지식 정보를 구축한다.

### Document data 로딩

샘플 코드에서 사용할 데이타 파일은 단순 텍스트 파일이며 git clone으로 생성된 local repository에서 이 노트북이 존재하는 위치의 아래의 data 폴더에 저장되어있다.

In [25]:
if run_in_cp4d:

    from ibm_watson_studio_lib import access_project_or_space
    wslib = access_project_or_space()

    # load data of type "text/plain" into a file like object
    # raw_data_0 = wslib.load_data('매드맥스-퓨리오사-사가.txt')
    wslib.download_file('퓨리오사-줄거리.txt')
    filename = '퓨리오사-줄거리.txt'
else:
    filename = os.path.join(os.getcwd(), 'data', '퓨리오사-줄거리.txt')

In [26]:
loader = TextLoader(filename, encoding='UTF8')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [27]:
len(texts)

21

In [28]:
texts[0:3]


[Document(metadata={'source': '/Users/jihun.kim/Documents/src/p-tect-genai-enablement/08-what-is-rag/data/퓨리오사-줄거리.txt'}, page_content='매드 맥스: 분노의 도로에서 쓰였던 배경 설명 대사들이 나오고 타이틀이 뜬 다음, 지구에서 오스트레일리아 대륙의 황무지 안에 있는 푸른 구역을 보여주는 장면이 나온다.\n퓨리오사는 동생 발키리와 함께 마을 구역에서 떨어진 장소에서 복숭아를 따먹던 중, 풍요의 땅을 발견하고 사냥을 마친 뒤 떠나려는 바이커 무리를 발견한다. 이들이 떠나면 풍요의 땅의 위치가 외부 세력에 발각 될 것을 우려한 퓨리오사는 동생인 발키리에게 조용히 있으라고 당부한 뒤 홀로 바이커 무리에 접근한다. 이야기를 나누는 바이커들 몰래 오토바이 한대의 연료 호스를 끊어내는 것에 성공했지만, 뒤에서 접근한 바이커에게 발각당하고 만다. 위기 상황에서 부발리니인 어머니, 메리 조 바사를 부르는 피리를 불었지만 결국 바이커들에게 제압당해 끌려가고 만다.[2]'),
 Document(metadata={'source': '/Users/jihun.kim/Documents/src/p-tect-genai-enablement/08-what-is-rag/data/퓨리오사-줄거리.txt'}, page_content='피리소리를 듣고 찾아온 바사가 다급하게 바이커 무리를 쫒아 대부분의 바이커를 죽이는 것에 성공하지만 두 명의 바이커는 빠져나가고 만다. 풍요의 땅이 존재한다는 증거로서 디멘투스에게 퓨리오사를 직접 진상하고 총애를 받겠다는 늙은 바이커[3]의 계획을 확인한 퓨리오사. 안장에 엎어져 실려가는 와중에도 이로 연료 호스를 물어뜯어 시간을 지연시켜 보았지만 결국 추격해온 바사가 마지막 바이커를 죽이지 못했고[4] 그대로 퓨리오사는 바이커 무리 한복판에 던져진다.'),
 Document(metadata={'source': '/Users/jihun.kim/Documents/s

### embedding function 생성

여기서는 huggingface에서 제공하는 기본 embedding 함수를 사용한다.
다른 embedding 함수를 사용할 수도 있으나 이 경우 embedding 함수와 사용하고자 하는 vector db (여기서는 Chroma)의 embedding size가 일치해야 한다.
참고로 Chroma db의 embedding size는 768 이다.

In [29]:
embeddings = HuggingFaceEmbeddings()

### Vector DB (Chroma db) 생성

chunk로 나누어진 데이터와 embeddng 모델을 사용하여 Chroma db를 생성한다. 이 함수가 실행될 때 각 chunk들이 embedding되고 vector화된 데이터가 원본 텍스트와 함께 db내에 저장된다.

In [30]:
docsearch = Chroma.from_documents(texts, embeddings)

<a id="models"></a>
## watsonx의 Foundation model 접근 설정

IBM watsonx의 foundation model들은 <a href="https://python.langchain.com/v0.2/docs/integrations/providers/ibm/#watsonxllm" target="_blank" rel="noopener no referrer">langchain에 의해 지원되는 LLM들의 목록</a>에 속한다.
여기서는 한글이 잘 동작한다고 평가되는 <a href="https://huggingface.co/mncai/llama2-13b-dpo-v7">mncai/llama2-13b-dpo-v7</a>를 사용한다.

WatsonxLLM class는 watsonx foundation model과 langchain 간의 interface를 제공해주는 class이다.

In [32]:
model_id = 'mncai/llama2-13b-dpo-v7'
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 500,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}
watsonx_llama2_korean = WatsonxLLM(
    model_id=model_id,
    url=wml_credentials.get("url"),
    username=wml_credentials.get("username"),
    apikey=wml_credentials.get("apikey"),
    instance_id=wml_credentials.get("instance_id"),
    project_id=project_id,
    params=parameters
)

Model 'mncai/llama2-13b-dpo-v7' is not supported for this environment. Supported models: []


WMLClientError: Model 'mncai/llama2-13b-dpo-v7' is not supported for this environment. Supported models: []

### 사용자 질의 검색 테스트

앞서 구축한 지식 기반 정보로부터 사용자의 질의와 가장 유사한 정보를 검색해서 출력해 본다.

In [ ]:
retriever = docsearch.as_retriever(search_kwargs={'k': 2})
resutls = retriever.get_relevant_documents("퓨리오사가 한 쪽 팔을 잃게되는 경위가 뭔가요?")
resutls

<a id="predict"></a>
## RAG를 사용하여 사용자 질문에 대한 답변을 생성한다.

langchain의 `RetrievalQA` (question answering chain) 를 사용하여 RAG 작업을 자동으로 수행한다.

In [ ]:
qa = RetrievalQA.from_chain_type(llm=watsonx_llama2_korean, chain_type="stuff", retriever=retriever)

In [ ]:
qa.invoke("퓨리오사가 한 쪽 팔을 잃게되는 경위가 뭔가요?")

In [ ]:
query = "등장인물 중 잭은 어떤 역할인가요?"
qa.run(query)

In [ ]:
qa.invoke("퓨리오사는 어떻게 어머니의 복수를 하나요?")